# Projeto Banco de Dados I
### Professor Rogério Mainardes
### Alunos Erick Henrique e Paulo Vitor 

Esse projeto consiste na análise descritiva e de 10 perguntas que serão respondidas através do uso de SQL para um dataset.
O dataset escolhido está disponível no [github](https://github.com/ValdisW/datasets/blob/master/video-game-sales.csv).

Os dados escolhidos mostram as vendas de jogos de videogame lançados entre 1980 e 2020 que tiveram mais de 100.000 vendas nas regiões América do Norte, União Europeia, Japão ou outros (resto do mundo).

Análise descritiva:

- Qual as tipagens das colunas?;
- Quantos nulos temos em cada coluna?;
- Quantos valores únicos temos nas colunas categoricas?;
- Existem outliers nos dados numéricos? Se sim, como lidar com eles?;

10 perguntas:

- Qual a soma de todos os jogos vendidos no Wii?
- Quais foram os jogos mais vendidos entre 2010 e 2016?
- Qual região foi responsável pela maior quantidade de vendas  de jogos?
- Quais as 3 plataformas com mais jogos vendidos?
- Quais os 10 jogos mais vendidos no japão?
- Quantos jogos foram lançados no ano de 2008?
- Quais as 3 plataformas com mais jogos lançados?
- A soma das vendas dos 3 jogo lançado em maior número de plataforma?

In [1]:
-- Criação da tabela videogame_sales no postgres
CREATE TABLE videogame_sales(
    ranking integer,
    game_name varchar(200),
    platform varchar(20),
    release_year smallint,
    genre varchar(20),
    publisher varchar(50),
    na_sales real,
    eu_sales real,
    jp_sales real,
    other_sales real,
    global_sales real
);

In [ ]:
-- cópia dos dados do arquivo videogame-sales para o postgres
COPY videogame_sales 
FROM 'caminho/do/arquivo/arquivo.csv'
WITH(
    FORMAT csv,
    DELIMITER ',', 
    HEADER,
    NULL 'N/A'
);

### Conectando com o banco de dados

In [4]:
# Instalando libs necessárias
!pip install sqlalchemy
!pip install pandas
!pip install pandasql

^C
ERROR: Operation cancelled by user


In [31]:
# Importa as libs instaladas

from sqlalchemy import create_engine
import pandas as pd
import pandasql as ps

In [70]:
# Configura conexão com o banco

SGBD = "postgresql+psycopg2"
USER = "postgres"
SENHA = "123"
HOST = "localhost"
DATABASE = "Video game dataset"

engine = create_engine(f"{SGBD}://{USER}:{SENHA}@{HOST}/{DATABASE}")
conn = engine.connect()

In [52]:
# Consulta teste
pd.read_sql_query(
    """
    SELECT 
        * 
    FROM 
        videogame_sales
    LIMIT 4
    """, 
    con=conn
)

,ranking,game_name,platform,release_year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00


### Análise Descritiva

#### Tipagem das colunas

In [62]:
pd.read_sql_query(
    """
    SELECT 
        column_name, data_type
    FROM 
        information_schema.columns
    WHERE
        table_schema = 'public' AND table_name = 'videogame_sales'
    """, 
    con=conn
)

,column_name,data_type
0,ranking,integer
1,game_name,character varying
2,platform,character varying
3,release_year,smallint
4,genre,character varying
5,publisher,character varying
6,na_sales,real
7,eu_sales,real
8,jp_sales,real
9,other_sales,real


Information schema é um schema criado automaticamente pelo postgres e que contém informações de todas as colunas no banco de dados. Filtramos esse schema para mostrar apenas dados da tabela que criamos (videogame_sales) e, especificamente, as colunas column_name e data_type.
Para verificar outras informações presentes nesse schema, altere o 'SELECT' para '*'.

#### Quantidade de valores nulos em cada coluna

In [71]:
pd.read_sql_query(
    """
    SELECT
	    COUNT(*) FILTER (WHERE ranking IS NULL) AS nulos_ranking,
	    COUNT(*) FILTER (WHERE game_name IS NULL) AS nulos_game_name,
        COUNT(*) FILTER (WHERE platform IS NULL) AS nulos_platform,
        COUNT(*) FILTER (WHERE release_year IS NULL) AS nulos_release_year,
        COUNT(*) FILTER (WHERE genre IS NULL) AS nulos_genre,
        COUNT(*) FILTER (WHERE publisher IS NULL) AS nulos_publisher,
        COUNT(*) FILTER (WHERE na_sales IS NULL) AS nulos_na_sales,
        COUNT(*) FILTER (WHERE eu_sales IS NULL) AS nulos_eu_sales,
        COUNT(*) FILTER (WHERE jp_sales IS NULL) AS nulos_jp_sales,
        COUNT(*) FILTER (WHERE other_sales IS NULL) AS nulos_other_sales,
        COUNT(*) FILTER (WHERE global_sales IS NULL) AS nulos_global_sales
    FROM
        videogame_sales
    """, 
    con=conn
)

,nulos_ranking,nulos_game_name,nulos_platform,nulos_release_year,nulos_genre,nulos_publisher,nulos_na_sales,nulos_eu_sales,nulos_jp_sales,nulos_other_sales,nulos_global_sales
0,0,0,0,271,0,58,0,0,0,0,0


A consulta acima conta todos os elementos presentes em uma coluna em que foi aplicado um filtro que mostra apenas os elementos de valor NULL, repetindo essa mesma contagem e aplicação de filtro para cada uma das colunas.
Assim, temos a contagem de elementos nulos em cada uma das colunas.

#### Valores únicos nas colunas de variáveis categóricas

Temos as seguintes colunas como representantes de variáveis categóricas:
- Plataforma
- Ano de lançamento
- Gênero
- Empresa desenvolvedora

A consulta abaixo pode nos mostrar o número de categorias distintas (valores únicos) em cada uma das colunas

In [ ]:
pd.read_sql_query(
    """
    SELECT 
	    count(DISTINCT platform) AS platform_count,
	    count(DISTINCT release_year) AS release_year_count,
	    count(DISTINCT genre) AS genre_count,
	    count(DISTINCT publisher) AS publisher_count
    FROM
        videogame_sales
    """, 
    con=conn
)

A consulta acima conta os valores distintos em cada uma das colunas e nos retorna essa contagem.

### 10 Perguntas